In [ ]:
import pandas as pd
Data = pd.read_csv('/content/drive/MyDrive/hotel review/train.csv')
Data.shape                 #(38932, 5)
Data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38932 entries, 0 to 38931
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User_ID       38932 non-null  object
 1   Description   38932 non-null  object
 2   Browser_Used  38932 non-null  object
 3   Device_Used   38932 non-null  object
 4   Is_Response   38932 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [ ]:
#Checking unique 
Data.describe().transpose()

,count,unique,top,freq
User_ID,38932,38932,id48154,1
Description,38932,38932,If asked there is no way that I would recommen...,1
Browser_Used,38932,11,Firefox,7367
Device_Used,38932,3,Desktop,15026
Is_Response,38932,2,happy,26521


In [ ]:
#Data Cleaning:
null_count = Data.isnull().sum().sort_values(ascending=False)
percentage = (Data.isnull().sum()/len(Data)*100).sort_values(ascending=False)
null_data = pd.concat([null_count,percentage],axis=1,keys=['null_count','percentage'])
null_data

,null_count,percentage
Is_Response,0,0.0
Device_Used,0,0.0
Browser_Used,0,0.0
Description,0,0.0
User_ID,0,0.0


In [ ]:
Data_Final = Data.drop(['User_ID', 'Browser_Used','Device_Used'], axis = 1)
Data_Final

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,not happy
1,I stayed at the Crown Plaza April -- - April -...,not happy
2,I booked this hotel through Hotwire at the low...,not happy
3,Stayed here with husband and sons on the way t...,happy
4,My girlfriends and I stayed here to celebrate ...,not happy
...,...,...
38927,We arrived late at night and walked in to a ch...,happy
38928,The only positive impression is location and p...,not happy
38929,Traveling with friends for shopping and a show...,not happy
38930,The experience was just ok. We paid extra for ...,not happy


In [ ]:
Description = Data_Final['Description']

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
# Text cleaning and create TF-IDF model
import re # regularization library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
corpus=[]
for i in range(len(Description)):
  review = re.sub('[^a-zA-z]', ' ', Description[i]) # to remove elements in paragraph other than a-z ,A-Z case words.
  review = review.lower()                          # Lowerize the sentences and  stoping keywords such as 'he', 'it' and split it into words
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)


In [ ]:
Data_Final['corpus'] = pd.DataFrame(corpus)
Data_Final.head()

,Description,Is_Response,corpus
0,The room was kind of clean but had a VERY stro...,not happy,room kind clean strong smell dog generally ave...
1,I stayed at the Crown Plaza April -- - April -...,not happy,stayed crown plaza april april staff friendly ...
2,I booked this hotel through Hotwire at the low...,not happy,booked hotel hotwire lowest price could find g...
3,Stayed here with husband and sons on the way t...,happy,stayed husband son way alaska cruise loved hot...
4,My girlfriends and I stayed here to celebrate ...,not happy,girlfriend stayed celebrate th birthday planne...


In [ ]:
# Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, Data_Final['Is_Response'], test_size=0.2, random_state=0)
print(len(X_train),len(X_test),len(y_train),len(y_test))

31145 7787 31145 7787


In [ ]:
#Train pipeline model:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
TdidfV = TfidfVectorizer()
LR = LogisticRegression()
Model = Pipeline([('vectorizer',TdidfV),('LogisticReg',LR)])
Model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('LogisticReg',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, inte

In [ ]:
#Evaluate the model:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
predictions = Model.predict(X_test)
print(confusion_matrix(predictions,y_test))
print(classification_report(predictions,y_test))
print(accuracy_score(predictions,y_test))

[[4976  585]
 [ 309 1917]]
              precision    recall  f1-score   support

       happy       0.94      0.89      0.92      5561
   not happy       0.77      0.86      0.81      2226

    accuracy                           0.89      7787
   macro avg       0.85      0.88      0.86      7787
weighted avg       0.89      0.89      0.89      7787

0.8851932708360087


In [ ]:
# Check the real output:
example = ['Room is not clean']
result = Model.predict(example)
result

array(['happy'], dtype=object)